In [19]:
from threading import Thread
import time 

def simple_worker():
    print(threading.current_thread())
    time.sleep(0.2)
    print("Bonjour")

t1 = Thread(target=simple_worker)
t1.start()


<Thread(Thread-20 (simple_worker), started 36720)>
Bonjour


In [20]:
from threading import Thread
import time 

def simple_worker():
    print(threading.current_thread())
    time.sleep(0.2)
    print("Bonjour")

t1 = Thread(target=simple_worker)
t1.start()
t1.start()

<Thread(Thread-21 (simple_worker), started 39844)>


RuntimeError: threads can only be started once

Bonjour


In [21]:
def info(thread):
    print(thread.name)
    print(thread.ident)
    print(thread.is_alive())


t1 = Thread(target=simple_worker)
info(t1)
t1.start()
info(t1)
time.sleep(1)
info(t1)



Thread-22 (simple_worker)
None
False
<Thread(Thread-22 (simple_worker), started 272)>
Thread-22 (simple_worker)
272
True
Bonjour
Thread-22 (simple_worker)
272
False


In [32]:
#environ 10 secondes 
def worker(caractere):
    for i in range(0,10):
        print(caractere, flush = True, end='')
        time.sleep(1)   
        
t_asterix = Thread(target=worker, args=("*",))
t_asterix.start()
t_point = Thread(target=worker, args=(".",))
t_point.start()

t_asterix.join()
t_point.join()

*..**..*.*.*.*.**..*

In [33]:
#environ 20 secondes 
class Benchmark:
    def __enter__(self):
        self.debut = time.time()
         
    def __exit__(self, exc_type, exc_instance, traceback):
        print(f"time is {time.time() - self.debut}")
        return True
with Benchmark():        
    worker("*")
    worker("*")

********************time is 20.025364637374878


In [34]:
#environ 10 secondes 
with Benchmark():        
    t_asterix = Thread(target=worker, args=("*",))
    t_asterix.start()
    t_point = Thread(target=worker, args=(".",))
    t_point.start()
    
    t_asterix.join()
    t_point.join()

*..**.*..*.**..*.**.time is 10.03180742263794


In [44]:
#correction 1
def ecrire_fichier():
    name = threading.current_thread().name
    for i in range(0, 11):
        with open("./compte.txt", "a") as f:
             f.write(name +" : "+ str(i)+"\n")
             f.flush()
threads = []
for i in range(0,11):    
    t = Thread(target=ecrire_fichier, name = f"Thread {i+1}")
    threads.append(t)
for t in threads:
    t.start()




In [46]:
#environ 10 secondes 
def worker(caractere):
    for i in range(0,10):
        print(caractere, flush = True, end='')
        time.sleep(1)   
        
t_asterix = Thread(target=worker, args=("*",))
t_asterix.start()

time.sleep(5)
t_underscore = Thread(target=worker, args=("_",))
t_underscore.start()

t_point = Thread(target=worker, args=(".",))
t_asterix.join()

t_point.start()


*****_*_*_*_*_*_._._._._......

In [52]:
#correction 2
def ecrire_fichier(iterater):
    name = threading.current_thread().name
    for i in iterater:
        with open("./compte.txt", "a") as f:
             f.write(name +" : "+ str(i)+"\n")
             f.flush()
threads_nombre = []
for i in range(0,3):    
    t = Thread(target=ecrire_fichier, args=(range(0, 11),), name = f"Thread {i+1}")
    threads_nombre.append(t)
    
threads_letter = []
for i in range(0,3):    
    t = Thread(target=ecrire_fichier, args=(iter("ABCDE"),), name = f"Thread {i+1}")
    threads_letter.append(t)
for t in threads_nombre:
    t.start()
for t in threads_nombre:
    t.join()    
for t in threads_letter:
    t.start()    


In [53]:
def worker(caractere):
    for i in range(0,10):
        print(caractere, flush = True, end='')
        time.sleep(1)   

class Caracter(Thread):
    def __init__(self, caractere, **kwargs):
        self.caractere = caractere
        super().__init__(**kwargs)

    def run(self):
        for i in range(0,10):
            print(self.caractere, flush = True, end='')
            time.sleep(1)   


t_asterix = Caracter("*")
t_asterix.start()
t_point = Caracter(".")
t_point.start()


*..**..**..*.**.*.*.

In [59]:
#corrrection classe
def ecrire_fichier(filepath, msg):
    name = threading.current_thread().name
    for i in range(0, 11):
        with open(filepath, "a") as f:
             f.write(name +" : "+ msg +"\n")
             f.flush()

class AjouterMessageThread(Thread):
    def __init__(self, filepath, msg, **kwargs):
        self.filepath = filepath
        self.msg = msg
        super().__init__(**kwargs)

    def run(self):
        name = threading.current_thread().name
        time.sleep(2)
        with open(self.filepath, "a") as f:
             f.write(name +" : "+ self.msg +"\n")
             f.flush()
threads = []
for i in range (0,3):
    threads.append(AjouterMessageThread("message.txt", f"Le {i+1} Msg"))
for t in threads:
    t.start()

for t in threads:
    t.join()

with open("message.txt", "r") as f:
    print(f.read())

    

Thread-58 : Le 1 Msg
Thread-60 : Le 3 Msg
Thread-61 : Le 1 Msg
Thread-62 : Le 2 Msg



In [66]:
from random import randint

class SharedData:
    value = None

def show_value(data):
    try:
        val = data.value 
    except AttribueError:
        print(threading.current_thread(), "Pas encore de valeur")
    else:
        print(threading.current_thread(), "- value : ", val)

def worker(data):
    show_value(data)
    data.value = randint(1, 1000)
    show_value(data)

shared_data = SharedData()

show_value(shared_data)
t1 = Thread(target = worker, args = (shared_data,))
t1.start()
t2 = Thread(target = worker, args = (shared_data,))
t2.start()
time.sleep(2)
show_value(shared_data)
    


<_MainThread(MainThread, started 32540)> - value :  None
<Thread(Thread-73 (worker), started 36300)> - value :  None
<Thread(Thread-73 (worker), started 36300)> - value :  796
<Thread(Thread-74 (worker), started 39644)> - value :  796
<Thread(Thread-74 (worker), started 39644)> - value :  294
<_MainThread(MainThread, started 32540)> - value :  294


In [69]:
from random import randint
from threading import local
class SharedData:
    value = None

def show_value(data):
    try:
        val = data.value 
    except AttributeError as e:
        print(threading.current_thread(), "Pas encore de valeur")
    else:
        print(threading.current_thread(), "- value : ", val)

def worker(data, default_value):
    data.value = default_value
    show_value(data)
    data.value = randint(1, 1000)
    show_value(data)

local_data = local()
local_data.value = 0 

show_value(local_data)
t1 = Thread(target = worker, args = (, default_value = 0 ))
t1.start()
t2 = Thread(target = worker, args = (local_data,default_value = 0))
t2.start()
time.sleep(2)
show_value(local_data)

<_MainThread(MainThread, started 32540)> Pas encore de valeur
<Thread(Thread-75 (worker), started 20784)> Pas encore de valeur
<Thread(Thread-75 (worker), started 20784)> - value :  765
<Thread(Thread-76 (worker), started 29964)> Pas encore de valeur
<Thread(Thread-76 (worker), started 29964)> - value :  585
<_MainThread(MainThread, started 32540)> Pas encore de valeur
